# Анализ базы данных сервиса для чтения книг

Цель исследования: проанализировать базу данных крупного сервиса для чтения книг по подписке.
В сервисе — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Данные помогут сформулировать ценностное предложение для нового продукта.

**Задачи, которые предстоит выполнить:**
1. [Исследовать таблицы.](#section_id_1)
2. [Выполнить по одному SQL-запросу для решения каждого задания:](#section_id_2)
* 2.1. Посчитать, сколько книг вышло после 1 января 2000 года.
* 2.2. Для каждой книги посчитать количество обзоров и среднюю оценку.
* 2.3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.
* 2.4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками.
* 2.5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
3. [Описать выводы по каждой из решённых задач.](#section_id_3)

**Описание данных**

Таблица `books` содержит данные о книгах:
* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

Таблица `authors` содержит данные об авторах:
* `author_id` — идентификатор автора;
* `author` — имя автора.

Таблица `publishers` содержит данные об издательствах:
* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства.

Таблица `ratings` содержит данные о пользовательских оценках книг:
* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

Таблица `reviews` содержит данные о пользовательских обзорах:
* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя автора обзора;
* `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## 1. Исследование таблиц.<a id='section_id_1'></a>

* **books**

In [2]:
query = '''SELECT * FROM books LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


* **authors**

In [3]:
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


* **publishers**

In [4]:
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


* **ratings**

In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


* **reviews**

In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## 2. SQL-запросы для решения каждого задания.<a id='section_id_2'></a>

**2.1. Посчитать, сколько книг вышло после 1 января 2000 года.**

In [7]:
query = '''SELECT COUNT(book_id) 
            FROM books 
            WHERE publication_date > '2000-1-1'
            '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


**2.2. Для каждой книги посчитать количество обзоров и среднюю оценку.**

In [8]:
query = '''WITH table_1 AS (SELECT book_id, AVG(rating) AS avg_rat
                            FROM ratings
                            GROUP BY book_id),
                table_2 AS (SELECT book_id, COUNT(review_id) AS count_rev
                            FROM reviews
                            GROUP BY book_id)
            SELECT b.title AS title,
                  b.book_id AS book_id,
                  COALESCE(t_2.count_rev, 0) AS count,
                  t_1.avg_rat AS avg
            FROM books AS b
            LEFT JOIN table_1 AS t_1 ON t_1.book_id=b.book_id
            LEFT JOIN table_2 AS t_2 ON t_2.book_id=b.book_id
            ORDER BY count DESC'''
pd.io.sql.read_sql(query, con = engine)

,title,book_id,count,avg
0,Twilight (Twilight #1),948,7,3.662500
1,The Road,854,6,3.772727
2,The Alchemist,627,6,3.789474
3,The Curious Incident of the Dog in the Night-Time,695,6,4.081081
4,The Giver (The Giver #1),733,6,3.750000
...,...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
996,Disney's Beauty and the Beast (A Little Golden...,191,0,4.000000
997,Essential Tales and Poems,221,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667


**2.3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц.**

Так мы исключаем из анализа брошюры.

In [9]:
query = '''WITH table_1 AS (SELECT COUNT(book_id) AS count_book,
                                 publisher_id
                          FROM books
                          WHERE num_pages > 50
                          GROUP BY publisher_id)
            SELECT p.publisher,
                   COUNT(book_id) AS count_book
            FROM publishers AS p
            LEFT JOIN books AS b ON p.publisher_id=b.publisher_id
            LEFT JOIN table_1 AS t_1 ON t_1.publisher_id=b.publisher_id
            GROUP BY p.publisher
            ORDER BY count_book DESC
            LIMIT 1'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_book
0,Penguin Books,42


**2.4. Определить автора с самой высокой средней оценкой книг — учитывайть только книги с 50 и более оценками**

In [10]:
query = '''WITH table_1 AS (SELECT b.book_id AS book_id, 
                                   b.author_id AS author_id,
                                   COUNT(rat.rating_id) AS count_rating
                            FROM books AS b
                            LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
                            GROUP BY b.book_id),
                table_2 AS (SELECT book_id,
                                   author_id,
                                   count_rating
                            FROM table_1
                            WHERE count_rating > 49),
                table_3 AS (SELECT t_2.book_id, AVG(rat.rating) AS avg_rating
                            FROM table_2 AS t_2
                            INNER JOIN ratings AS rat ON t_2.book_id=rat.book_id
                            GROUP BY t_2.book_id)
            SELECT a.author, AVG(t_3.avg_rating) AS avg
            FROM authors AS a
            INNER JOIN books AS b ON a.author_id = b.author_id
            INNER JOIN table_3 AS t_3 ON b.book_id = t_3.book_id
            GROUP BY a.author
            ORDER BY avg DESC
            LIMIT 1'''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


**2.5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

In [11]:
query = '''WITH table_1 AS (SELECT username, COUNT(rating) AS count
                          FROM ratings
                          GROUP BY username
                          HAVING COUNT(rating) > 50),
               table_2 AS (SELECT r.username, COUNT(review_id) AS count
                           FROM reviews AS r
                           INNER JOIN table_1 AS t ON r.username = t.username
                           GROUP BY r.username)
               SELECT ROUND(AVG(count))
               FROM table_2'''
pd.io.sql.read_sql(query, con = engine)

,round
0,24.0


## 3. Выводы.<a id='section_id_3'></a>

* После 1 января 2000 года вышло 819 книг. Реклама приложения может помочь не только читателям заинтересоваться ресурсом, но и современным авторам опубликовать свои шедевры.
* Для каждой книги посчитано количество обзоров и средняя оценка. Введение дополнительных критериев может помочь пользователям лучше ориентироваться на сервисе. 
* Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц. Сотрудничество с издательствами поможет значительно расширить область книг и их жанров.
* J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг (при учёте книг с 50 и более оценками). Популярность киниги о Гарри Поттере может оказать хорошую рекламу сервису, в котором без труда можно найти все сборники в одном месте.
* Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24. Бесплатный пробный период - рабочий вариант увеличения спроса любого сервиса. Увеличение спроса позволит повысить количество заинтересованных пользователей, у которых будет желание делиться своим мнением в обзорах книг.